In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
students = pd.DataFrame([1,1])

def selectData(students: pd.DataFrame) -> pd.DataFrame:
    return students.shape

selectData(students)

(2, 1)

In [3]:
print(selectData(students))

(2, 1)


In [4]:
def createBonusColumn(students: pd.DataFrame) -> pd.DataFrame:
    students['bonus'] = students['salary'] * 2
    return students

In [7]:
students = pd.DataFrame([(1,1),(2,1)])

In [9]:
selectData(students)

(2, 2)

In [10]:
def selectData(students: pd.DataFrame) -> pd.DataFrame:
    return students.shape

In [11]:
selectData(students)

(2, 2)